# Aula 10 - Visconde

Leandro Carísio Fernandes


Implementar um pipeline multidoc QA: dado uma pergunta do usuário, buscamos em uma grande coleção as passagens mais relevantes e as enviamos para um sistema agregador, que irá gerar uma resposta final.

- Avaliar no dataset do IIRC
- Métrica principal: F1
- Limitar dataset de teste para 50 exemplos para economizar.
- Usar o gpt-3.5-turbo como modelo agregador. Usar vicuna-13B como alternativa open-source:

 - https://huggingface.co/helloollel/vicuna-13b
 - https://chat.lmsys.org/

Dicas:

- Se inspirar no pipeline do Visconde: https://github.com/neuralmind-ai/visconde



## Setup - variáveis, instalação de libs, conexão com o Drive etc

In [1]:
# Se True, segmenta os artigos e salva num pickle. Se False, apenas recupera o pickle do drive
gerar_artigos_segmentados = False
arquivo_artigos_segmentados = 'conteudo_artigos_segmentados.pickle'

# Se True, gera o índice invertido e salva em um pickle. Se False, apenas recupera o pickle do drive
gerar_indice_invertido = False
arquivo_indice_invertido = 'iidx_artigos_segmentados' # A função salva em 2 arquivos, então ela mesma coloca a extensão pickle

# Se True, faz o rankeamento (nesse ponto é bom ter a GPU ligada, senão vai demorar umas 3-4 horas pra fazer apenas na CPU) e salva
# Se False, recupera o reranking do arquivo pickle
gerar_segmentos_relevantes_para_queries = False
arquivo_seg_relevantes_para_queries = 'seg_relevantes_para_queries.pickle'
batch_size = 1024 # Para reranking (1024 precisa de menos de 11 GB de RAM pro minilm)

# Se True, faz a agregação usando GPT-3.5-turbo e salva. Se False, recupera do arquivo pickle
agregar_usando_gpt35turbo = False
arquivo_respostas_geradas = 'respostas_geradas_usando_gpt3.5-turbo-few-shot.pickle'

dir_aula10 = "/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula10-visconde/"

# Não estou conseguindo usar o MonoT5. Vou testar com o classificador binário da aula 2
modelo_classificador_binario = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula2-classificador-binario-mini-bert/'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyserini -q
!pip install faiss-cpu -q
!pip install transformers -q
!pip install openai -q

## Download da base de dados

Link para a base IIRC: https://github.com/jferguson144/IIRC-baseline

Baixa arquivo iirc.tar.gz e context_articles.tar.gz conforme setup.sh do IIRC: https://github.com/jferguson144/IIRC-baseline/blob/main/setup.sh

In [4]:
!wget http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
!tar -xzf iirc.tar.gz
!rm iirc.tar.gz
!cd iirc

!wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
!tar -xzf context_articles.tar.gz
!rm context_articles.tar.gz

!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json 

--2023-05-17 19:02:46--  http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
Resolving jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)... 52.218.242.243, 52.92.227.17, 52.92.229.249, ...
Connecting to jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)|52.218.242.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5713947 (5.4M) [application/x-gzip]
Saving to: ‘iirc.tar.gz’

iirc.tar.gz         100%[===================>]   5.45M  10.6MB/s    in 0.5s    

2023-05-17 19:02:47 (10.6 MB/s) - ‘iirc.tar.gz’ saved [5713947/5713947]

--2023-05-17 19:02:47--  http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
Resolving jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)... 52.218.242.243, 52.92.227.17, 52.92.229.249, ...
Connecting to jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)|52.218.242.243|:80... connected.
HTTP request sent, awai

In [5]:
import json

dev_set = json.load(open('iirc/dev.json','r'))
test_set = json.load(open('iirc_test.json', 'r'))
context_articles = json.load(open("context_articles.json",'r'))

Entendendo os dados de teste. Daqui dá pra pegar 50 questões e respostas pra gente testar depois que tiver o resultado.

In [6]:
# Imprime as 50 primeiras perguntas e a sua resposta esperada. E já salva em um array de objetos
# Observação: Se mudar isso daqui (por exemplo, de test_set pra dev_set) é necessário
# regerar o arquivo arquivo_seg_relevantes_para_queries (ou seja, setar gerar_segmentos_relevantes_para_queries = True)
# e rodar o caderno com um GPU ligada. 
perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]['questions'][0]
  
  pergunta = pr['question']
  resposta_obj = pr['answer']
  tipo_resposta = resposta_obj['type']

  if tipo_resposta == 'binary' or tipo_resposta == 'value':
    resposta = resposta_obj['answer_value']
  elif tipo_resposta == 'span':
    resposta = resposta_obj['answer_spans'][0]['text']
  elif tipo_resposta == 'none':
    resposta = 'none'
  else:
    resposta = 'Não pode chegar aqui, os tipos parece que são só binary/value/span/none!'
    print(tipo_resposta)
  
  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta})

  if i < 50:
    print(f"{i}: {pergunta}\n{resposta}")
    print('----------------------------------------------------------------------')

0: What is Zeus know for in Greek mythology?
sky and thunder god
----------------------------------------------------------------------
1: How long had the First World War been over when Messe was named aide-de-camp?
5
----------------------------------------------------------------------
2: How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
2
----------------------------------------------------------------------
3: What is the capacity of the stadium where Brunt returned to action after a torn ACL?
26,688
----------------------------------------------------------------------
4: In which country was Wilhelm Müller born?
Germany 
----------------------------------------------------------------------
5: In which Italian region did Pesce studied medicine?
Liguria
----------------------------------------------------------------------
6: What albums were ranked higher than "It Takes a Nation of Millions to Hold Us Back" i

## Indexa pra rodar um BM25

Remove o html da coleção:

In [7]:
import re

# Remover o html igual está na base IIRC é bem mais rápido que usar o BeuatifulSoup
# https://github.com/jferguson144/IIRC-baseline/blob/main/util.py
def remove_html(html):
  return re.sub("<[^>]*>", "", html).strip()

In [8]:
%%time
documents = []
all_titles = []

for item in test_set:
    if item['title'].lower() not in all_titles:
        documents.append({
                "title": item['title'],
                "content": item["text"]
            }
        )
        all_titles.append(item['title'].lower())
    for link in item["links"]:
        if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
            documents.append({
                "title": link['target'],
                "content": context_articles[link['target'].lower()]
            })
            all_titles.append(link['target'].lower())
        # else:
            # print(link['target'].lower())

CPU times: user 727 ms, sys: 6.15 ms, total: 733 ms
Wall time: 797 ms


In [9]:
documents[0]

{'title': 'Palici',
 'content': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld. There was also a shrine to the Palaci in Palacia, where people could subject themselves or others to tests of reliability through divine judgement; passing meant that an oath could be trusted. The mythological lineage of the Palici is uncertain; one legend made the Palici the sons of Zeus, or possibly Hephaestus, by Aetna or Thalia, but another claimed that the Palici were the sons of the Sicilian deity Adranus.\n"}

In [10]:
artigos_sem_html = []

# Se for usar todos os context_articles para indexar:
#for titulo in context_articles.keys():
#  artigo_html = context_articles[titulo]
#  artigos_sem_html.append({"title": titulo, "content": remove_html(artigo_html)})

# Se for usar só os documentos que estão na base de teste
for doc in documents:
  titulo = doc['title']
  conteudo = doc['content']
  artigos_sem_html.append({"title": titulo, "content": remove_html(conteudo)})

Quebra cada artigo em grupos de 3 sentenças. Usa a biblioteca Spacy para separar as sentenças.

Código de origem: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb

In [11]:
%%time
from tqdm import tqdm
import spacy
import pickle

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['title'],
          "contents": document['title']+". "+segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


if gerar_artigos_segmentados:
  artigos_segmentados = window(artigos_sem_html)
  with open(f'{dir_aula10}{arquivo_artigos_segmentados}', 'wb') as f:
    pickle.dump(artigos_segmentados, f)
else:
  with open(f'{dir_aula10}{arquivo_artigos_segmentados}', 'rb') as f:
    artigos_segmentados = pickle.load(f)

CPU times: user 11.5 s, sys: 4.82 s, total: 16.3 s
Wall time: 17 s


In [12]:
len(artigos_segmentados)

498082

Define classes para índice invertido e BM25:

In [13]:
%%time
from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.lucene_analyser = Analyzer(get_lucene_analyzer(stemmer='porter'))

  def tokenizar(self, texto):
    return self.lucene_analyser.analyze(texto)

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])   
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 966 ms, sys: 145 ms, total: 1.11 s
Wall time: 1.47 s


Agora efetivamente indexa os artigos segmentados:

In [14]:
def salvar_indice_em_arquivo_pickle(indice_invertido):
  nome_arquivo_base = f"{dir_aula10}{arquivo_indice_invertido}"

  with open(f'{nome_arquivo_base}_indice.pickle', 'wb') as f:
    pickle.dump(indice_invertido.indice, f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'wb') as f:
    pickle.dump(indice_invertido.tamanho_doc, f)

def recuperar_indice_em_arquivo_pickle():
  nome_arquivo_base = f"{dir_aula10}{arquivo_indice_invertido}"
  idx = IndiceInvertido()

  with open(f'{nome_arquivo_base}_indice.pickle', 'rb') as f:
    idx.indice = pickle.load(f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'rb') as f:
    idx.tamanho_doc = pickle.load(f)
  idx.n_docs = len(idx.tamanho_doc)

  return idx

In [15]:
%%time
iidx = IndiceInvertido()

if gerar_indice_invertido:
  # Vamos usar o próprio índice como um id para o segmento. Vai facilitar a vida depois
  # art_seg['contents'] tem o título e 3 frases do artigo
  for id, art_seg in enumerate(artigos_segmentados):
    iidx.adiciona_doc(id, art_seg['contents'])

    if id % 50000 == 0:
      print(f'{id} segmentos indexados')

  salvar_indice_em_arquivo_pickle(iidx)
else:
  iidx = recuperar_indice_em_arquivo_pickle()

CPU times: user 5.07 s, sys: 2.87 s, total: 7.93 s
Wall time: 8.27 s


Agora sim cria o buscador BM25 e testa:

In [16]:
buscador = BM25(iidx, 0.82, 0.68, 1)

Faz um teste pesquisando a primeira pergunta e exibindo os 10 primeiros resultados do BM25. É interessante ver que só usar o BM25 não resolve o problema, visto que nesse caso os 10 primeiros resultados nem tem o resultado esperado. O reranking é mesmo uma necessidade!

In [17]:
print('Pergunta e resposta esperada: ')
print(perg_resposta_esperada[0])

resultado_bm25 = buscador.pesquisar(perg_resposta_esperada[0]['pergunta'])

print('-----------------------------------------------------------------')
print('10 primeiros conteúdos retornados pelo BM25\n')
for i in range(10):
  segmento_id = resultado_bm25[i][0]
  score = resultado_bm25[i][1]

  title = artigos_segmentados[segmento_id]['title']
  contents = artigos_segmentados[segmento_id]['contents']
  segment = artigos_segmentados[segmento_id]['segment']

  print(f'Segmento_id: {segmento_id}. Score: {score}')
  print(f"Title: {title}")
  print(f"Contents: {contents}")
  print('-----------------------------------------------------------------')

Pergunta e resposta esperada: 
{'pergunta': 'What is Zeus know for in Greek mythology?', 'resposta': 'sky and thunder god'}
-----------------------------------------------------------------
10 primeiros conteúdos retornados pelo BM25

Segmento_id: 1140. Score: 26.877893447875977
Title: Zeus
Contents: Zeus. - Rohde, Erwin, Psyche: The Cult of Souls and Belief in Immortality among the Greeks, 1925. - Smith, William, Dictionary of Greek and Roman Biography and Mythology, 1870, Ancientlibrary.com, William Smith, Dictionary: "Zeus" Ancientlibrary.com

External links.- Greek Mythology Link, Zeus stories of Zeus in myth
- Theoi Project, Zeus summary, stories, classical art
- Theoi Project, Cult Of Zeus cult and statues
- Photo: Pagans Honor Zeus at Ancient Athens Temple from National Geographic
-----------------------------------------------------------------
Segmento_id: 510. Score: 25.476608276367188
Title: Greek mythology
Contents: Greek mythology. In Ancient Roman times, a new Roman mytho

## Reranking dos resultados

Código adaptado do Visconde: https://github.com/neuralmind-ai/visconde/blob/main/qasper_rerank.ipynb

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(modelo_classificador_binario)
model = AutoModelForSequenceClassification.from_pretrained(modelo_classificador_binario).to(device)

Como não consegui instalar o PyGaggle, vou me inspirar no código do PyGaggle pra tentar fazer o reranking com o monoT5:

In [19]:
from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

class Dataset(data.Dataset):
  def __init__(self, tokenizer, query, textos_para_rankear, max_seq_length = 256):
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length    
    self.lista_query_textos = [f'{query} [SEP] {t}' for t in textos_para_rankear]

  def __len__(self):
    return len(self.lista_query_textos)
    
  def __getitem__(self, idx):
    batch_encoding = self.tokenizer.encode_plus(self.lista_query_textos[idx],
                                                add_special_tokens=True,
                                                truncation=True,
                                                max_length=self.max_seq_length,
                                                padding='max_length',
                                                return_attention_mask=True,
                                                return_tensors='pt')
    return {
      'input_ids': batch_encoding['input_ids'].squeeze(0),
      'attention_mask': batch_encoding['attention_mask'],
    }

@torch.no_grad()
def pesquisa_e_rerank(query):
  # Pesquisa
  docs_score = buscador.pesquisar(query)

  # Pega apenas os 1000 primeiros elementos e extrai o conteúdo deles
  docs_score = docs_score[:1000]
  doc_ids, _ = zip(*docs_score)
  doc_contents = []
  for idx, id in enumerate(doc_ids):
    doc_contents.append(artigos_segmentados[id]['contents'])

  # Cria um dataset pra mandar pro reranking
  dataset_reranking = Dataset(tokenizer, query, doc_contents)
  # Agora cria um dataloader
  dataloader_reranking = DataLoader(dataset_reranking, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

  # Agora sim, faz o reranking
  scores = []
  for batch in tqdm(dataloader_reranking, desc=f'Reranking para query: {query}', disable=False):
    outputs = model(**batch.to(device))
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    _, classe1 = zip(*probs)
    scores.extend(list(classe1))
  
  scores_float = [t.item() for t in scores]
  docs_scores = sorted(zip(doc_ids, scores_float), key=lambda x: x[1], reverse=True)

  return docs_scores

Vamos pesquisar as 50 primeiras queries e guardar os resultados:

In [20]:
%%time
segmentos_relevantes_por_query = {}

if gerar_segmentos_relevantes_para_queries:
  for i in range(len(perg_resposta_esperada)):
    print(f'Query: {i}')
    docs_scores = pesquisa_e_rerank(perg_resposta_esperada[i]['pergunta'])
    segmentos_relevantes_por_query[i] = docs_scores

  with open(f'{dir_aula10}{arquivo_seg_relevantes_para_queries}', 'wb') as f:
    pickle.dump(segmentos_relevantes_por_query, f)
else:
  with open(f'{dir_aula10}{arquivo_seg_relevantes_para_queries}', 'rb') as f:
    segmentos_relevantes_por_query = pickle.load(f)



CPU times: user 95.1 ms, sys: 64.1 ms, total: 159 ms
Wall time: 166 ms


In [21]:
# Verifica o resultado para alguma query específica
# Pode ser de 0 a 513

idx_query = 502
print(perg_resposta_esperada[idx_query]['pergunta'])
print(f'Resposta esperada: {perg_resposta_esperada[idx_query]["resposta"]}')
print('------------------------------------')
print('TOP 5 documentos retornados:')
print('------------------------------------')


for i in range(10):
  docs_scores = segmentos_relevantes_por_query[idx_query]
  print(f'{i} {artigos_segmentados[docs_scores[i][0]]}')

Does the writer of the song "Who Did That to You" have more than two children?
Resposta esperada: no
------------------------------------
TOP 5 documentos retornados:
------------------------------------
0 {'title': 'Dream Theater', 'contents': 'Dream Theater. The whole band substantially reworked the song, and it appeared on the album as "You Not Me" with a chorus that bore little resemblance to the original. Child also had a noticeable impact on the album, with a shift towards less complex and more radio-friendly compositions. The band wrote almost two CDs worth of material, including a 20-minute-long follow-up to the Images and Words song "Metropolis–Part I: The Miracle and the Sleeper".', 'segment': 'The whole band substantially reworked the song, and it appeared on the album as "You Not Me" with a chorus that bore little resemblance to the original. Child also had a noticeable impact on the album, with a shift towards less complex and more radio-friendly compositions. The band wro

## Etapa de agregação com GPT-3.5-turbo



Vamos selecionar algumas perguntas/respostas pra gente passar no prompt do GPT-3.5:

In [22]:
idx_query = 305

pergunta = perg_resposta_esperada[idx_query]['pergunta']
resposta_esperada = perg_resposta_esperada[idx_query]["resposta"]
print(pergunta)
print(f'Resposta esperada: {resposta_esperada}')

for i in range(5):
  docs_scores = segmentos_relevantes_por_query[idx_query]
  print(artigos_segmentados[docs_scores[i][0]]['segment'])
print('-------------------------------------------------------')

When was Article Two of the US Constitution drafted?
Resposta esperada: September 17
The electors cast their votes on the Monday following the second Wednesday in December (the first Monday after December 12) of that year. Thereafter, the votes are opened and counted by the vice president, as president of the Senate, in a joint session of Congress. Clause 5: Qualifications for office.
When President Bill Clinton attempted to shield the records of the President's Task Force on Health Care Reform as essential to his functions under the Recommendations Clause, a federal circuit court rejected the argument and noted in Ass'n of American Physicians & Surgeons v. Clinton (1993): "[T]he Recommendation Clause is less an obligation than a right. The president has the undisputed authority to recommend legislation, but he need not exercise that authority with respect to any particular subject or, for that matter, any subject." Clause 3: Convening and adjourning of Congress.
A quorum in the House 

In [23]:
artigos_segmentados[docs_scores[1][0]]['segment']

'When President Bill Clinton attempted to shield the records of the President\'s Task Force on Health Care Reform as essential to his functions under the Recommendations Clause, a federal circuit court rejected the argument and noted in Ass\'n of American Physicians & Surgeons v. Clinton (1993): "[T]he Recommendation Clause is less an obligation than a right. The president has the undisputed authority to recommend legislation, but he need not exercise that authority with respect to any particular subject or, for that matter, any subject." Clause 3: Convening and adjourning of Congress.'

In [24]:
idx_query = 99
print(perg_resposta_esperada[idx_query]['pergunta'])
print(f'Resposta esperada: {perg_resposta_esperada[idx_query]["resposta"]}')
print('------------------------------------')
print('TOP 5 documentos retornados:')
print('------------------------------------')


for i in range(5):
  docs_scores = segmentos_relevantes_por_query[idx_query]
  print(f'{i} {artigos_segmentados[docs_scores[i][0]]}')

On what date did the Loma Prieta earthquake occur?
Resposta esperada: 1989-10-18
------------------------------------
TOP 5 documentos retornados:
------------------------------------
0 {'title': '1989 Loma Prieta earthquake', 'contents': '1989 Loma Prieta earthquake. The 1989 Loma Prieta earthquake occurred on California’s Central Coast on October 17 at local time (1989-10-18 00:04 UTC). The shock was centered in The Forest of Nisene Marks State Park approximately 10 mi northeast of Santa Cruz on a section of the San Andreas Fault System and was named for the nearby Loma Prieta Peak in the Santa Cruz Mountains. With an magnitude of 6.9 and a maximum Modified Mercalli intensity of IX (Violent), the shock was responsible for 63 deaths and 3,757 injuries.', 'segment': 'The 1989 Loma Prieta earthquake occurred on California’s Central Coast on October 17 at local time (1989-10-18 00:04 UTC). The shock was centered in The Forest of Nisene Marks State Park approximately 10 mi northeast of Sa

In [25]:
import os
import openai

# Pra rodar no Colab tem que colocar a chave aqui.
openai.api_key = os.getenv('API_KEY_OPENAI')

def mensagem_user(numExemplo, pergunta, lista_documentos):
  doc_com_id = [ f"[Document {i+1}]: {lista_documentos[i]}" for i in range(len(lista_documentos))]
  return f"Example {numExemplo}\n\n" + "\n\n".join(doc_com_id) + f"\n\nQuestion: {pergunta}"

def mensagem_assistant(evidencia, resposta):
  return f"Evidence: {evidencia}\n\nAnswer: {resposta}"

few_shot_examples = [
    {"user": "", "assistant": ""},
    {"user": "", "assistant": ""},
]
few_shot_examples[0]["user"] = mensagem_user(1, "Did Pink Floyd have a song about the French Riviera?", [
    "\"San Tropez\" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band's \"best of\" album, Echoes: The Best of Pink Floyd.",
    "The French Riviera (known in French as the Côte d'Azur [kot daˈzyʁ]; Occitan: Còsta d'Azur [ˈkɔstɔ daˈzyɾ]; literal translation \"Azure Coast\") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d'Azur (Région Sud) region of France. The Principality of Monaco is a semi-enclave within the region, surrounded on three sides by France and fronting the Mediterranean.",
    "Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.",
    "Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz , pronounced [san(t) tʀuˈpes]) is a town on the French Riviera, 68 kilometres (42 miles) west of Nice and 100 kilometres (62 miles) east of Marseille in the Var department of the Provence-Alpes-Côte d'Azur region of Occitania, Southern France."
])
few_shot_examples[0]["assistant"] = mensagem_assistant("\"San Tropez\" is a song by Pink Floyd about the French Riviera [Document 1]. Saint-Tropez is a town on the French Riviera [Document 4]", "yes")
few_shot_examples[1]["user"] = mensagem_user(2, "On what date did the Loma Prieta earthquake occur?", [
    "The 1989 Loma Prieta earthquake occurred on California’s Central Coast on October 17 at local time (1989-10-18 00:04 UTC). The shock was centered in The Forest of Nisene Marks State Park approximately 10 mi northeast of Santa Cruz on a section of the San Andreas Fault System and was named for the nearby Loma Prieta Peak in the Santa Cruz Mountains. With an magnitude of 6.9 and a maximum Modified Mercalli intensity of IX (Violent), the shock was responsible for 63 deaths and 3,757 injuries.",
    "'Foreshocks. The 5.3 June 1988 and the 5.4 August 1989 events also occurred on previously unknown oblique reverse faults and were within 3 mi of the M6.9 Loma Prieta mainshock epicenter, near the intersection of the San Andreas and Sargent faults. Total displacement for these shocks was relatively small (approximately 4 in of strike-slip and substantially less reverse-slip) and although they occurred on separate faults and well before the mainshock, a group of seismologists considered these to be foreshocks due to their location in space and time relative to the main event.'",
    "The June 27, 1988, shock occurred with a maximum intensity of VI (Strong). Its effects included broken windows in Los Gatos, and other light damage in Holy City, where increased flow was observed at a water well. Farther away from the Santa Cruz Mountains, pieces of concrete fell from a parking structure at the Sunnyvale Town Center, a two-level shopping mall in Santa Clara County.",
    "The 6.9 Loma Prieta earthquake occurred on October 17, 1989. The rupture was related to the San Andreas fault system and affected the entire San Francisco Bay Area with a maximum Mercalli intensity of IX (Violent). Many structures in Oakland were badly damaged including the double-decker portion of Interstate 880 that collapsed.",
])
few_shot_examples[1]["assistant"] = mensagem_assistant("According to [Document 1] and [Document 4], the Loma Prieta earthquake occur on October 17, 1989", "1989-10-18")

def agrega(query = "", docs_para_agregar = []):
  response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
              messages=[
                  {"role": "system", "content": "You are a helpful reader that reads a list of documents and use them as evidence to answer a question. You provide detailed explanations and concise answers."},
                  {"role": "user", "content": "For each example, write an explanation to the answer given the documents and the question. The final answer should be short (less than 10 words)."},
                  {"role": "user", "content": few_shot_examples[0]["user"]},
                  {"role": "assistant", "content": few_shot_examples[0]["assistant"]},
                  {"role": "user", "content": few_shot_examples[1]["user"]},
                  {"role": "assistant", "content": few_shot_examples[1]["assistant"]},
                  {"role": "user", "content": mensagem_user(2, query, docs_para_agregar)}
              ],
              temperature=0,
              max_tokens=1000)
  return response['choices'][0]['message']['content']

In [26]:
total_docs = 5

respostas_llm = {}

if agregar_usando_gpt35turbo:
  for idx_query in range(100):
    query = perg_resposta_esperada[idx_query]['pergunta']
    resposta_esperada = perg_resposta_esperada[idx_query]['resposta']
    docs_scores = segmentos_relevantes_por_query[idx_query]
    # índice 0 é o id do segmento, índice 1 é o score
    textos = [ artigos_segmentados[docs_scores[idx_doc][0]]['segment'] for idx_doc in range(total_docs)]

    respostas_llm[idx_query] = agrega(query, textos)

    print(f'Query {idx_query}: {query}')
    print(f'Resposta esperada: {resposta_esperada}')
    print(f'Resposta LLM: {respostas_llm[idx_query]}')

  with open(f'{dir_aula10}{arquivo_respostas_geradas}', 'wb') as f:
    pickle.dump(respostas_llm, f)
else:
  with open(f'{dir_aula10}{arquivo_respostas_geradas}', 'rb') as f:
    respostas_llm = pickle.load(f)

## Cálculo do F1

In [27]:
# Funções normize_answer e token_f1_score foram retiradas do Visconde (arquivo qasper_evaluator.py)
import re
import string

def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def extrai_resposta(resp):
  idx_resposta_llm = resp.find('Answer: ') + 8
  return resp[idx_resposta_llm:]

# Trata respostas vazias do GPT-3.5
def uniformiza_resposta_vazia(resp):
  valores_possiveis = ["n/a", "it is not possible to determine", "no information in the documents", "no information provided", "none of the documents", "documents do not provide", "unknown", "cannot be answered", "not mentioned", "not specified", "n/a", "not available", "insufficient data", "not enough information", "cannot be determined", "there is no mention"]
  
  if resp == '':
    return 'none'

  resp = resp.lower()

  for valor_none in valores_possiveis:
    if valor_none in resp:
      return 'none'
  
  return resp

In [28]:
import pandas as pd

col_queries = []
col_resp_esperadas = []
col_respostas_llm = []
col_f1_score = []

for idx_query in respostas_llm:
  query = perg_resposta_esperada[idx_query]['pergunta']
  resposta_esperada = normalize_answer(perg_resposta_esperada[idx_query]['resposta'])
  resposta_llm = normalize_answer(uniformiza_resposta_vazia(extrai_resposta(respostas_llm[idx_query])))
  f1_score = token_f1_score(resposta_llm, resposta_esperada)

  #print(f'{query}\n{resposta_esperada}\n{resposta_llm}\n---------------------------------------------')

  col_queries.append(query)
  col_resp_esperadas.append(resposta_esperada)
  col_respostas_llm.append(resposta_llm)
  col_f1_score.append(f1_score)

df = pd.DataFrame({'Query': col_queries, 'Resp_esperada': col_resp_esperadas, 'Resposta_LLM': col_respostas_llm, 'F1_score': col_f1_score})
df

,Query,Resp_esperada,Resposta_LLM,F1_score
0,What is Zeus know for in Greek mythology?,sky and thunder god,god of sky and thunder,0.888889
1,How long had the First World War been over whe...,5,none,0.000000
2,How long had Angela Scoular been acting profes...,2,2 years,0.666667
3,What is the capacity of the stadium where Brun...,26688,none,0.000000
4,In which country was Wilhelm Müller born?,germany,germany,1.000000
...,...,...,...,...
95,How far did Schooley travel from his hometown ...,none,none,1.000000
96,How old was the artist whose song Isa Raja san...,36,none,0.000000
97,What other ships were in the fleet with the Dr...,none,none,1.000000
98,How many years after Chess Records was founded...,2,2 years,0.666667


In [29]:
# Dados com resposta esperada
df_com_resp_esperada = df[df['Resp_esperada'] != 'none']

print(len(df_com_resp_esperada))

69


In [30]:
# Exact match
print('Considerando todas as questões')
print(sum(df['Resp_esperada'] == df['Resposta_LLM'])/len(df))
print('\nConsiderando só questões que tem uma resposta esperada')
print(sum(df_com_resp_esperada['Resp_esperada'] == df_com_resp_esperada['Resposta_LLM'])/len(df_com_resp_esperada))

Considerando todas as questões
0.31

Considerando só questões que tem uma resposta esperada
0.15942028985507245


In [31]:
# F1 score
print('Considerando todas as questões')
print(df['F1_score'].mean())
print('\nConsiderando só questões que tem uma resposta esperada')
print(df_com_resp_esperada['F1_score'].mean())

Considerando todas as questões
0.3777222222222222

Considerando só questões que tem uma resposta esperada
0.2575684380032206
